In [1]:
# Set up the configuration for accessing the storage account
storage_account_name = "team12storage"
storage_account_key = "ZdpCqxRwuQbJ8fUYN7EMbTLjOyj+I9rI6IjcQ/uU+4aFG46TQqjiz1Piq9SWKGkkjszo3r94l26M+AStR5PvPg=="
container = "team12blobcontainer"
abfss_base_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/"
df = spark.read.parquet(f"{abfss_base_path}Medallion/Gold/dataset_updated/")

StatementMeta(stockPredict, 42, 2, Finished, Available, Finished)

In [2]:
from pyspark.sql.functions import max as spark_max, avg, first
user_profile_df = df.groupBy("user_id").agg(
    first("is_blue_verified").alias("is_blue_verified"),
    first("account_created_at").alias("account_created_at"),
    spark_max("account_age_days").alias("account_age_days"),
    first("is_new_account").alias("is_new_account"),
    first("is_influencer").alias("is_influencer"),
    spark_max("followers_count").alias("followers_count"),
    spark_max("friends_count").alias("friends_count"),
    spark_max("account_favourites_count").alias("account_favourites_count"),
    spark_max("listed_count").alias("listed_count"),
    spark_max("media_count").alias("media_count"),
    avg("credibility_score").alias("credibility_score"),
    avg("follower_activity_score").alias("follower_activity_score")
)

display(user_profile_df.limit(10))

StatementMeta(stockPredict, 36, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b686570b-ab08-427f-a9d0-517a740ac74a)

In [3]:
from pyspark.sql.functions import avg, sum as spark_sum, count, col

daily_tweet_summary = df.groupBy("tweet_created_at_date").agg(
    count("*").alias("total_tweets"),
    avg("sentiment_score").alias("avg_sentiment"),
    avg("interaction_score").alias("avg_interaction_score"),
    avg("favorite_ratio").alias("avg_favorite_ratio"),
    avg("reply_ratio").alias("avg_reply_ratio"),
    spark_sum(col("is_viral").cast("int")).alias("total_viral_tweet"),
    avg("follower_activity_score").alias("avg_activity_score"),
    avg("credibility_score").alias("avg_credibility_score"),
    avg("followers_count").alias("avg_followers")

)
display(daily_tweet_summary.limit(10))

StatementMeta(stockPredict, 36, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5603d0f7-c1e7-4f66-bb90-c8a03538e4dc)

In [2]:
predicted_df = spark.read.parquet(f"{abfss_base_path}Prediction/Batch/data/")

StatementMeta(stockPredict, 42, 3, Finished, Available, Finished)

In [4]:
from pyspark.sql.functions import avg, sum as spark_sum, count, col
market_summary_per_day = df.groupBy("tweet_created_at_date").agg(
    avg("current_open").alias("current_open"),
    avg("current_close").alias("current_close"),
    avg("current_high").alias("current_high"),
    avg("current_low").alias("current_low"),
    avg("current_volume").alias("current_volume"),
    avg("next_available_volume").alias("actual_volume")
)
display(market_summary_per_day.limit(10))

StatementMeta(stockPredict, 42, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f87f4450-7380-4fe3-a71e-b225312352ba)

In [5]:
predicted_df = predicted_df.withColumnRenamed("predicted_next_day_volume", "predicted_volume")
market_summary_per_day = market_summary_per_day.join(
    predicted_df.select("tweet_created_at_date", "predicted_volume"),
    on="tweet_created_at_date",
    how="left"
)
display(market_summary_per_day.limit(10))

StatementMeta(stockPredict, 42, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0bd79ece-2c8d-4b9e-8c3d-2c084384fdd2)

In [7]:
viral_tweet_log = df.filter(col("is_viral") == True).select(
    "user_id", "tweet_created_at_date", "full_text",
    "sentiment_score", "interaction_score", "followers_count",
    "favorite_count", "retweet_count", "reply_count", "view_count", "credibility_score"
)
display(viral_tweet_log.limit(10))

StatementMeta(stockPredict, 35, 8, Finished, Available, Finished)

In [8]:
from pyspark.sql.functions import sum as spark_sum, count, col, countDistinct

user_type_distribution_per_day = df.groupBy("tweet_created_at_date").agg(
    spark_sum(col("is_influencer").cast("int")).alias("influencer_tweet_count"),
    spark_sum(col("is_new_account").cast("int")).alias("new_account_tweet_count"),
    spark_sum(col("is_blue_verified").cast("int")).alias("blue_verified_tweet_count"),
    count("*").alias("total_tweets"),
    countDistinct("user_id").alias("distinct_users"),
    avg("account_age_days").alias("avg_account_age")
)
display(user_type_distribution_per_day.limit(10))

StatementMeta(stockPredict, 35, 9, Finished, Available, Finished)

In [9]:
from pyspark.sql.functions import desc

# Total tweets per user
top_users_by_tweet_count = df.groupBy("user_id").agg(
    count("*").alias("tweet_count")
).orderBy(desc("tweet_count"))

# Viral tweets per user
top_users_by_viral_tweets = df.filter(col("is_viral") == True).groupBy("user_id").agg(
    count("*").alias("viral_tweet_count")
).orderBy(desc("viral_tweet_count"))

display(top_users_by_tweet_count.limit(10))
display(top_users_by_viral_tweets.limit(10))

StatementMeta(stockPredict, 35, 10, Finished, Available, Finished)

In [10]:
from pyspark.sql.functions import month, year

influencer_monthly_engagement = df.filter(col("is_influencer") == True).groupBy(
    year("tweet_created_at_date").alias("year"),
    month("tweet_created_at_date").alias("month")
).agg(
    count("*").alias("influencer_tweet_count"),
    avg("sentiment_score").alias("avg_sentiment_score"),
    avg("interaction_score").alias("avg_interaction"),
    countDistinct("user_id").alias("distinct_influencers")
)
display(influencer_monthly_engagement.limit(10))

StatementMeta(stockPredict, 35, 11, Finished, Available, Finished)

In [6]:
# Save aggregated data as single files
abfss_base_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/Medallion/Gold"
daily_tweet_summary.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/daily_tweet_summary")
viral_tweet_log.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/viral_tweet_log")
user_type_distribution_per_day.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/user_type_distribution_per_day")
top_users_by_tweet_count.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/top_users_by_tweet_count")
top_users_by_viral_tweets.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/top_users_by_viral_tweets")
influencer_monthly_engagement.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/influencer_monthly_engagement")
market_summary_per_day.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/market_summary_per_day")
user_profile_df.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/user_profile")

StatementMeta(stockPredict, 42, 7, Finished, Available, Finished)